**Configuration** <br>
* Single Class : PV_all
* Learning rate : lr0=0.0001

In [ ]:
yaml_content = """
path: /home/il72/cape_town_year_of_installation/datasets/pv_capetown_after_qc_5K
train: images/train
val: images/val
test: images/test

names:
  0: PV_all
"""

with open("/home/il72/cape_town_year_of_installation/YOLO_CapeTown_5K_single_categories(July_14)/data.yml", "w") as f:
    f.write(yaml_content)

print("data.yml (with path) created")


In [ ]:
# Optional: Clear CUDA cache and set GPU (0 or 1)
import os
import torch
torch.cuda.empty_cache()

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
print("Using device:", torch.cuda.get_device_name(0)) 

**Training**

In [ ]:
from ultralytics import YOLO

# Load your pretrained model
# model = YOLO("yolo11x.pt")  # you already have this file
# model = YOLO("yolo11x-obb.pt")
model = YOLO("yolo11x-obb.pt")

# Train with your own dataset
model.train(
    data="/home/il72/cape_town_year_of_installation/YOLO_CapeTown_5K_single_categories(July_14)/data.yml",  # your dataset config file
    task="obb",
    epochs=100,                    # or any number that fits your case 100
    imgsz=1024,                     # image size, Must corresponds with input image size
    device=0,                      # mps for Mac, 0 for 
    batch=2,                      # adjust depending on your VRAM
    lr0=0.0001,                # initial learning rate
    name="pv_detection_5K_yolo11x_obb_single_class_lr0.0001"    # experiment name
)

Evaluaton (Generating `.txt` for visualization)

In [2]:
import torch
from ultralytics import YOLO

# Manually set CUDA device to GPU 1
torch.cuda.set_device(1)

from ultralytics import YOLO

# Load your trained model (not the base pretrained model!)
model = YOLO("/home/il72/cape_town_year_of_installation/YOLO_CapeTown_5K_single_categories(July_14)/runs/obb/pv_detection_5K_yolo11x_obb_single_class_lr0.0001/weights/best.pt")

# Run evaluation on test set with proper data config
results = model.val(
    data="/home/il72/cape_town_year_of_installation/YOLO_CapeTown_5K_single_categories(July_14)/data.yml",
    split="test",
    task="obb",  # Make sure it runs in oriented bounding box mode
    save=True,          # Save prediction visualizations
    save_txt=True,      # Save prediction coordinates
    save_conf=True      # Save confidence values (important for analysis)
)

# Print summary metrics
print("Test Evaluation Results:")
print(f"mAP@0.5: {results.box.map50:.4f}")
print(f"mAP@0.5:0.95: {results.box.map:.4f}")

# Per-class metrics
print("\nPer-class metrics:")
for i, class_name in enumerate(results.names):
    print(f"{class_name} -> Precision: {results.box.p[i]:.4f}, Recall: {results.box.r[i]:.4f}, mAP@0.5: {results.box.maps[i]:.4f}")

Ultralytics 8.3.157 🚀 Python-3.10.12 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 3080, 10001MiB)
YOLO11x-obb summary (fused): 199 layers, 58,736,758 parameters, 0 gradients, 202.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 6100.6±3745.8 MB/s, size: 3075.5 KB)


val: Scanning /data/users/il72/cape_town_year_of_installation/datasets/pv_capetown_after_qc_5K/labels/test.cache... 249 images, 0 backgrounds, 0 corrupt: 100%|██████████| 249/249 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:12<00:00,  1.30it/s]


                   all        249       1136      0.836      0.798      0.825      0.703
Speed: 1.2ms preprocess, 38.4ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/obb/val6
Test Evaluation Results:
mAP@0.5: 0.8249
mAP@0.5:0.95: 0.7032

Per-class metrics:
0 -> Precision: 0.8359, Recall: 0.7981, mAP@0.5: 0.7032
